In [88]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
np.core.arrayprint._line_width = 800


In [196]:
import tensorflow as tf
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [239]:
# dataset=ImageDataGenerator()
# image = dataset.flow_from_directory(parent,target_size=(28,28),save_to_dir=os.path.join(parent,'aug'),save_prefix='aug',batch_size=10240)
# image.next()

In [234]:
# Custom augmentation
parent = '../../data/mnistAugment'
xTrain = []
yTrain = []
xTest = []
yTest = []
for subdir in os.listdir(parent):
    path = os.path.join(parent,subdir)
    print('\n',path)
    n = len(os.listdir(path))
    for i, file in enumerate(os.listdir(path)):
        print(i, end='\r')
        img = cv2.imread(os.path.join(path,file), cv2.IMREAD_GRAYSCALE)
        if i<(6/7)*n:
            xTrain.append(img)
            yTrain.append(int(path[-2:]))
        else:
            xTest.append(img)
            yTest.append(int(path[-2:]))
    
    
xTrainAdd = np.stack(xTrain)
yTrainAdd = np.stack(yTrain)
xTestAdd = np.array(xTest)
yTestAdd = np.array(yTest)


 ../mnistAugment\10
5119
 ../mnistAugment\11
5119
 ../mnistAugment\12
5119
 ../mnistAugment\13


In [237]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.concatenate((x_train,xTrainAdd))
y_train = np.concatenate((y_train,yTrainAdd))
x_test = np.concatenate((x_test,xTestAdd))
y_test = np.concatenate((y_test,yTestAdd))

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255

In [238]:
# Importing the required Keras modules containing model and layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(14,activation=tf.nn.softmax))

In [244]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=10)

Epoch 1/10
2424/2424 [==============================] - 34s 14ms/step - loss: 0.0647 - accuracy: 0.9805
Epoch 2/10
2424/2424 [==============================] - 31s 13ms/step - loss: 0.0396 - accuracy: 0.9876
Epoch 3/10
2424/2424 [==============================] - 32s 13ms/step - loss: 0.0274 - accuracy: 0.9913
Epoch 4/10
2424/2424 [==============================] - 35s 14ms/step - loss: 0.0211 - accuracy: 0.9935
Epoch 5/10
2424/2424 [==============================] - 35s 15ms/step - loss: 0.0204 - accuracy: 0.9934
Epoch 6/10
2424/2424 [==============================] - 32s 13ms/step - loss: 0.0163 - accuracy: 0.9945
Epoch 7/10
2424/2424 [==============================] - 35s 14ms/step - loss: 0.0159 - accuracy: 0.9946
Epoch 8/10
2424/2424 [==============================] - 33s 14ms/step - loss: 0.0137 - accuracy: 0.9955
Epoch 9/10
2424/2424 [==============================] - 33s 14ms/step - loss: 0.0148 - accuracy: 0.9953
Epoch 10/10
2424/2424 [==============================] - 35s 14m

In [245]:
model.save('../model/charModel')

INFO:tensorflow:Assets written to: ../model/charModel\assets


In [242]:
def show(img, text='img'):
    cv2.imshow(str(text), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [243]:
for sub in os.listdir(parent):
    print()
    print(sub)
    for file in os.listdir(os.path.join(parent,sub)):
        img = cv2.imread(os.path.join(parent,sub,file), cv2.IMREAD_GRAYSCALE)

        scaleFactor = 20/max(img.shape[:2])
        width = int(img.shape[1] * scaleFactor)
        height = int(img.shape[0] * scaleFactor)
        dim = (width, height)

        scaled = cv2.resize(img,dim)

        canvas = np.zeros((28,28), dtype = "uint8")

        x = int((28 - scaled.shape[1])/2)
        y = int((28 - scaled.shape[0])/2)
        canvas[y:y+scaled.shape[0],x:x+scaled.shape[1]] = scaled

        print(model.predict(canvas.reshape(1, 28, 28, 1)))


10
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.

KeyboardInterrupt: 